In [1]:
#read in audit file, using ausearch to interpret numeric values
from subprocess import Popen, PIPE
p = Popen(['ausearch','-if','example-data/audit.log','-i'],stdout=PIPE,stderr=PIPE)
out, err = p.communicate()

lines = out.split('\n')
print 'Log line count (pre-filter): ', len(lines)

Log line count (pre-filter):  3333


In [2]:
lines[:10]

['----',
 'type=DAEMON_START msg=audit(12/01/2016 13:03:32.540:2343) : auditd start, ver=2.4.5 format=raw kernel=4.4.0-21-generic auid=unset pid=11502 subj=unconfined  res=success ',
 '----',
 "type=USER_END msg=audit(12/01/2016 13:03:35.140:26) : pid=10875 uid=root auid=unset ses=unset msg='op=PAM:session_close acct=root exe=/usr/bin/sudo hostname=? addr=? terminal=/dev/pts/0 res=success' ",
 '----',
 "type=CRED_DISP msg=audit(12/01/2016 13:03:35.140:27) : pid=10875 uid=root auid=unset ses=unset msg='op=PAM:setcred acct=root exe=/usr/bin/sudo hostname=? addr=? terminal=/dev/pts/0 res=success' ",
 '----',
 'type=SECCOMP msg=audit(12/01/2016 13:04:05.182:28) : auid=unset uid=sampedro gid=sampedro ses=unset pid=11567 comm=chrome exe=/opt/google/chrome/chrome sig=SIG0 arch=x86_64 syscall=set_robust_list compat=0 ip=0x7f94507dc694 code=errno ',
 '----',
 'type=SECCOMP msg=audit(12/01/2016 13:04:05.186:29) : auid=unset uid=sampedro gid=sampedro ses=unset pid=11570 comm=chrome exe=/opt/googl

In [10]:
# parse lines into events
import re

evt_pattern = r"type=(\w+) msg=audit\((.+):([0-9]+)\) : (.*)"

events = {}
for l in lines:
    mgrp = re.match(evt_pattern,l.strip())
    
    if not mgrp:
        continue
        
    evt_type = mgrp.group(1)
    evt_date = mgrp.group(2)
    evt_id = mgrp.group(3)
    
    if evt_id not in events:
        events[evt_id] = {
            'date': evt_date
        }
    
    evt = events[evt_id]
    evt[evt_type] = {}
    evt_cmp = evt[evt_type]
    evt_data = mgrp.group(4)
    
    if evt_type == 'PROCTITLE':
        k, v = evt_data.split('=', 1)
        evt_cmp[k] = v
    else:
        for d in evt_data.split():
            try:
                k, v = d.split('=', 1)
                evt_cmp[k] = v
            except:
                pass
    
print len(events.keys()), ' events logged.'

1634  events logged.


In [27]:
events['1321']

{'CWD': {'cwd': '/etc/audit'},
 'PATH': {'item': '0', 'name': '/usr/sbin/pager', 'nametype': 'UNKNOWN'},
 'PROCTITLE': {'proctitle': 'systemctl status auditd'},
 'SYSCALL': {'a0': '0x7ffdd8ab236d',
  'a1': '0x7ffdd8ab2430',
  'a2': '0x55e83ed34010',
  'a3': '0x7fe8f824d7c0',
  'arch': 'x86_64',
  'auid': 'unset',
  'comm': 'systemctl',
  'egid': 'root',
  'euid': 'root',
  'exe': '/bin/systemctl',
  'exit': '-2(No',
  'fsgid': 'root',
  'fsuid': 'root',
  'gid': 'root',
  'items': '1',
  'key': '(null)',
  'pid': '14115',
  'ppid': '14114',
  'ses': 'unset',
  'sgid': 'root',
  'success': 'no',
  'suid': 'root',
  'syscall': 'execve',
  'tty': 'pts3',
  'uid': 'root'},
 'date': '12/01/2016 13:54:09.465'}

In [30]:
for k,v in events.iteritems():
    try:
        if ('USER_START' in v.keys()) or ('USER_END' in v.keys()):
            print k
            print v
    except:
        pass

50
{'date': '12/01/2016 13:04:28.319', 'USER_START': {'auid': 'unset', 'exe': '/usr/bin/pkexec', 'ses': 'unset', 'uid': 'sampedro', 'res': "success'", 'hostname': '?', 'pid': '11602', 'terminal': '?', 'msg': "'op=PAM:session_open", 'acct': 'root', 'addr': '?'}}
26
{'date': '12/01/2016 13:03:35.140', 'USER_END': {'auid': 'unset', 'exe': '/usr/bin/sudo', 'ses': 'unset', 'uid': 'root', 'res': "success'", 'hostname': '?', 'pid': '10875', 'terminal': '/dev/pts/0', 'msg': "'op=PAM:session_close", 'acct': 'root', 'addr': '?'}}
668
{'date': '12/01/2016 13:28:14.476', 'USER_START': {'auid': 'unset', 'exe': '/usr/bin/sudo', 'ses': 'unset', 'uid': 'root', 'res': "success'", 'hostname': '?', 'pid': '12864', 'terminal': '/dev/pts/3', 'msg': "'op=PAM:session_open", 'acct': 'root', 'addr': '?'}}
672
{'date': '12/01/2016 13:28:14.484', 'USER_START': {'auid': 'unset', 'exe': '/bin/su', 'ses': 'unset', 'uid': 'root', 'res': "success'", 'hostname': '?', 'pid': '12865', 'terminal': '/dev/pts/3', 'msg': "'

In [19]:
import datetime
import pytz
from django.utils import timezone

date_str = '12/01/2016 13:28:14.476'

date = datetime.datetime.strptime(date_str,'%m/%d/%Y %H:%M:%S.%f')
date = pytz.timezone('America/Denver').localize(date)
print date

2016-12-01 13:28:14.476000-07:00
